# Tutorial: VisionKG - A Data-Centric Way to Train your own Obejct Detection Models

# 1. QuickView of VisionKG

## 1.1 One-Click to meet VisionKG

In [ ]:
%load_ext autoreload
%autoreload 2

from google.colab import output
# install our vision api
!python -m pip install git+https://github.com/cqels/vision.git --force
output.clear()

## 1.2 Query a Dataset as YOU need via VisionKG

In [ ]:
# import SemkgAPI
from vision_utils import semkg_api, data
from skimage import io
import matplotlib.pyplot as plt

query_string='''#Give me 100 images containing car and truck
prefix cv:<http://vision.semkg.org/onto/v0.1/>
SELECT DISTINCT ?image
WHERE {
    ?ann1 a cv:Annotation.
    ?ann1 cv:isAnnotationOfImage ?image.
    ?ann1 cv:hasAnnotatedObject ?obj1.
    ?obj1 cv:hasLabel "car".
    ?ann2 a cv:Annotation.
    ?ann2 cv:isAnnotationOfImage ?image.
    ?ann2 cv:hasAnnotatedObject ?obj2.
    ?obj2 cv:hasLabel "truck".
    ?image cv:hasLocalPath ?localPath.
}
LIMIT 100'''

#Query and return result
result=semkg_api.query(query_string)


#Display sample images
rows=3
cols=4
f, ax_arr = plt.subplots(rows, cols, figsize=(16,8))
for j, row in enumerate(ax_arr):
    for i, ax in enumerate(row):
        if j*cols+i < len(result['images']):
            image = io.imread(semkg_api.SEMKG_IMAGES_HOST + result['images'][j*cols+i]['image_path'])
            ax.imshow(image)

f.suptitle("Sample images from the query result", fontsize=16)
plt.show()

# 2. Object Detection in Practice starting from VisionKG

## 2.1 Prepare and set parameters for training and evaluation

In [ ]:
from torch_model_zoo import utils
from os.path import join as opj

path_to_anno_mixedDatasets = opj('data/mixedDatasets/','test_query_api_image.json')
params = utils.prepare_for_training(path_to_anno_mixedDatasets, result)

##2.2 Dataset Playground

In [ ]:
if num_categories > 4:
  cat_nms = nms_categories[0:4]
else:
  cat_nms = nms_categories
  
utils.show_annotation(path_to_anno_mixedDatasets, cat_nms, show_num=6)
utils.show_cat_distribution(path_to_anno_mixedDatasets, cat_nms)

## 2.3 Perform Training & Evaluation on pytorch build-in models

In [ ]:
from torch_model_zoo.train_eval import train_eval_pipeline

train_eval_pipeline(params)